# Importando Bibliotecas e módulos necessários

In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'src')))
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dss import dss
from powerflow.batery import distbess_dictionary
from powerflow.generators import distgen_dictionary,add_gd
from powerflow.get_powers import get_dist_power
from powerflow.load import load_dictionary,add_load
from powerflow.text_comands import set_frequency,new_circuit,line_geometry,add_line,add_infos


# Diretórios dos dados


In [2]:
# Diretório de datasets
dir_datasets = 'C:\\Users\\joao9\\GitHub\\opendss\\datasets\\'

#Diretório de arquivos .dss
dir_dss = dir_datasets + 'arquivos_dss\\'

# Diretório de arquivos de geração PV
dir_pv = 'datasets\\generation_profiles\\'

# Diretório de arquivos de cargas
dir_cargas = 'datasets\\load_profiles\\'


# Definições iniciais do circuito

In [3]:

Text = dss.Text
Circuit = dss.ActiveCircuit
Solution = dss.ActiveCircuit.Solution

# Definições da MatPlotLib
plt.close('all')
plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (8,4)
plt.rcParams["figure.dpi"] = 100
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['font.sans-serif'] = ['cmr10']
plt.rcParams['axes.unicode_minus'] = False

#Potências máximas
PowerLoad=4
PowerPV=6 #Potência total dos módulos

# Definições do BESS
Pmin=0.320
BESS_P_MAX = 5
BESS_C_MAX = 12
SOC_MIN = 0.1 # 10%
SOC_MAX = 0.9 # 90%
BESS_E_MIN = BESS_C_MAX * SOC_MIN
BESS_E_MAX = BESS_C_MAX * SOC_MAX

# Definições do intervalo de simulação
nconsumidores = len(os.listdir(dir_cargas))
horas = 24 #hours
intervalo = 30 #min

# Criação do modelo base do arquivo.dss

In [4]:
with open(dir_dss + 'ModelagemTeste.dss', 'w') as file:
    file.write(set_frequency(60) + '\n')
    file.write(new_circuit('ModelagemTeste', str(1).zfill(3), 0.22) + '\n')
    file.write(line_geometry(0.52,0.63,'km',3.338,3,3,[-120.00,0.00,60.00],[1000.00,1000.00,1000.00])+ '\n')
    for i in range(1, nconsumidores+1,1):
        file.write(add_line(i,str(i).zfill(3),str(i+1).zfill(3),25,'m',3)+ '\n')
    
    file.write(add_infos())

opendssmodel = dir_dss + 'ModelagemTeste.dss'

# Criação dos dicionários que irão armazenar as inforamções das cargas, geraçãoPV e bateria

In [5]:
profiles = os.listdir(dir_cargas)

#Criando os dicionário com as cargas
loads ={}
for i in range(1,nconsumidores,1):
    bus = str(i).zfill(3)
    name = profiles[i].replace('.csv','')
    dir_profile = dir_cargas + profiles[i]
    loads[name] = load_dictionary(bus,3,0.22,PowerLoad,dir_profile)

# Criação do dicionário de geração PV   
distgen = {}
bus = '001'
name = 'pvbus001'
dir_profilePV = dir_pv + 'dados_power_normalizado.csv'
distgen[name] = distgen_dictionary(bus,3,0.22,PowerPV,dir_profilePV)

# Garantindo que o Opendss esteja limpo

In [6]:
dss.ClearAll()
dss.Start(0)
Text.Command = "Clear"
Text.Command = f"Compile {opendssmodel}"

bus_names = np.array(Circuit.AllBusNames)
print(bus_names)

['bus_001' 'bus_002' 'bus_003' 'bus_004' 'bus_005' 'bus_006' 'bus_007'
 'bus_008' 'bus_009' 'bus_010' 'bus_011' 'bus_012' 'bus_013' 'bus_014'
 'bus_015' 'bus_016' 'bus_017' 'bus_018' 'bus_019' 'bus_020' 'bus_021'
 'bus_022' 'bus_023' 'bus_024' 'bus_025' 'bus_026' 'bus_027' 'bus_028'
 'bus_029' 'bus_030' 'bus_031' 'bus_032' 'bus_033' 'bus_034' 'bus_035'
 'bus_036' 'bus_037' 'bus_038' 'bus_039' 'bus_040' 'bus_041']


# Fluxo de carga

In [13]:
bus_names = bus_names.reshape(bus_names.shape[0], 1)
column_power = ['time', 'line', 'bus_1', 'bus_2', 'P', 'Q']
power_df = pd.DataFrame(columns = column_power)

column_node_power = ['time', 'name', 'P', 'Q', 'P_profile', 'Q_profile']
node_power_df1 = pd.DataFrame(columns=column_node_power)

range_time = pd.date_range('2012-07-06 12:00:00', periods=horas*60/intervalo, freq=str(intervalo)+'T')
for time in range_time:
  new_command = ''
  for load in loads:
    new_command = new_command + '\n' + add_load(time, load, loads[load]['bus'], loads[load]['phases'], 
    loads[load]['voltage'], loads[load]['power'], loads[load]['profile'])

  for gen in distgen:
    new_command = new_command + '\n' + add_gd(time, gen, distgen[gen]['bus'], distgen[gen]['phases'], 
    distgen[gen]['conn'], distgen[gen]['voltage'], distgen[gen]['power'], distgen[gen]['profile'])



  dss.ClearAll()
  dss.Start(0)
  dss("Clear")
  dss(f"Compile {opendssmodel}")
  dss(new_command)
  dss("Set VoltageBases = [.22]")
  dss("CalcVoltageBases")
  dss("Set maxiterations = 500")
  dss('set mode = snap')
  dss("Solve")

  # Power Flow to DataFrame
  power = np.array(Circuit.PDElements.AllPowers)
  # print(power)
  power = power.reshape((-1, 6))
  # print(power)
  power = pd.DataFrame(power, columns=['P_a', 'Q_a', 'P_b', 'Q_b', 'P_c', 'Q_c'])
  # display(power)


  # Create new column that contains sum of specific columns
  power['P'] = power[['P_a', 'P_b', 'P_c']].sum(axis=1)
  power['Q'] = power[['Q_a', 'Q_b', 'Q_c']].sum(axis=1)

  # Get the index for each row of the Data Frame
  power_flow_index = []
  for line in Circuit.Lines:
      power_flow_index.append([time, line.Name, line.Bus1, line.Bus2])
      power_flow_index.append([time, line.Name, line.Bus1, line.Bus2])
  # print(power_flow_index)

  power[['time', 'line', 'bus_1', 'bus_2']] = power_flow_index
  # display(power)
  power_df = pd.concat([power_df, power])
  # display(power_df)


  # Get distributed generation power
  node_power = get_dist_power(column_node_power, time, power, loads, distgen)

  # display(node_power)

  # Add Grid and Losses power to DataFrame
  # The last "Power Balance" of the system, to be used in the smoothing algorithm.
  node_power = pd.concat([node_power, pd.DataFrame([[time, 'grid', -dss.Circuit.TotalPower()[0], -dss.Circuit.TotalPower()[1], 0, 0]], columns=column_node_power)])
  node_power = pd.concat([node_power, pd.DataFrame([[time, 'losses', dss.Circuit.Losses()[0]/1000, dss.Circuit.Losses()[1]/1000, 0, 0]], columns=column_node_power)])


  node_power_df1 = pd.concat([node_power_df1, node_power])


C:\Users\joao9\AppData\Local\Temp\ipykernel_22144\3777888131.py:8: FutureWarning: Non-integer 'periods' in pd.date_range, pd.timedelta_range, pd.period_range, and pd.interval_range are deprecated and will raise in a future version.
  range_time = pd.date_range('2012-07-06 12:00:00', periods=horas*60/intervalo, freq=str(intervalo)+'T')
C:\Users\joao9\AppData\Local\Temp\ipykernel_22144\3777888131.py:8: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  range_time = pd.date_range('2012-07-06 12:00:00', periods=horas*60/intervalo, freq=str(intervalo)+'T')
C:\Users\joao9\AppData\Local\Temp\ipykernel_22144\3777888131.py:54: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  power_df = pd.concat([power

KeyError: 0

# Extraindo os dados para o Plot

In [ ]:
# carga_kw1_SB = []
# circuit_kw1_SB =[]
# pv6_kw1_SB =[]
# bateria_kw1_SB = []
# losses_kw1_SB = []
# for timestep in range(loop-1):
#     # Filtrar as linhas onde `timestep` é o atual e `name` é `loadbus13` ou `loadbus02`
#     filtered_load = node_power_df1[(node_power_df1['timestep'] == timestep) & (node_power_df1['name'].isin(['loadbus021']))]
#     filtered_grid = node_power_df1[(node_power_df1['timestep'] == timestep) & (node_power_df1['name'].isin(['grid']))]
#     filtered_pv = node_power_df1[(node_power_df1['timestep'] == timestep) & (node_power_df1['name'].isin(['pvbus022']))]
#     filtered_bes = node_power_df1[(node_power_df1['timestep'] == timestep) & (node_power_df1['name'].isin(['bessbus023']))]
#     filtered_losses = node_power_df1[(node_power_df1['timestep'] == timestep) & (node_power_df1['name'].isin(['losses']))]

#     # Acessar a coluna `P` dessas linhas
#     p_values = sum(filtered_load['P'].values)
#     carga_kw1_SB.append(p_values)

#     grid_values = filtered_grid['P'].values[0]
#     circuit_kw1_SB.append(grid_values)

#     pv_values = filtered_pv['P'].values[0]
#     pv6_kw1_SB.append(pv_values)

#     # bess_values = filtered_bes['P'].values[0]
#     # bateria_kw1.append(bess_values)

#     losses_values = filtered_losses['P'].values[0]
#     losses_kw1_SB.append(losses_values)



# carga_kw_SB = np.array(carga_kw1_SB)
# bateria_kw = np.array(bateria_kw1_SB)
# pv6_kw_SB = -np.array(pv6_kw1_SB)
# circuit_kw_SB = -np.array(circuit_kw1_SB)
# losses_kw = np.array(losses_kw1_SB)

# Dmed_SB = np.mean(circuit_kw_SB)
# Dmax_SB = np.max(circuit_kw_SB)
# print('Demanda Média',Dmed_SB)
# print('Demanda Máxima',Dmax_SB)